# BrainShift     
This project investigate the longitudinal changes in brain position for macaques that are repeatedly scanned. This is done for a collaboration between the NIN and IMEC, aimed at developing wireless tools for brain recording and stimulation.

Chris Klink (c.klink@nin.knaw.nl)
Yao-Hong Liu (Yao-Hong.Liu@imec.nl)

## Requirements    
This Jupyter notebook runs `bash` using modulaes from [freesurfer](https://surfer.nmr.mgh.harvard.edu/) and [fsl](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSL)

## Strategy
The main approach is as follows.    

### Input
The `input` folder contains T1w scans from 2 monkey subjects acquired over a period of ~4 years.    

### Processing    
This notebook will take several steps to investigate potential shifts:
1) Normalize/uniformize all scans     
2) Generate the motion corrected average     
3) Align each unform scan to the average     
4) Generate concatenated 4d file without alignment       
5) Define a measure to look at
    - Discuss with Yao-Hong what would be a sensible measure    
    - Manual annotation seems inevitable

### Output    
Save the results in a sensible format

## Step 0: Configure     

In [ ]:
rootfld=$HOME'/Documents/MRI_ANALYSIS/NHP-BrainShift'
datafld=$HOME'/Dropbox/CURRENT_PROJECTS/NHP_MRI/Projects/BrainShift'

In [ ]:
#SUB='danny'
SUB='eddy'

## Step 1: Normalize individual scans    

In [ ]:
cd $datafld/input/sub-$SUB

# normalize individual scans
mkdir -p $datafld/normalized/sub-$SUB
for f in *.nii.gz; do
	echo $f
	mri_nu_correct.mni --i $f --o $datafld/normalized/sub-$SUB/nu_$f --distance 24
done 

## Step 2: Motion corrected average

In [ ]:
# get the motion corrected average
mkdir -p $datafld/aligned/sub-$SUB
cd $datafld/normalized/sub-$SUB
mri_motion_correct2 -o $datafld/aligned/sub-$SUB/avg_sub-$SUB.nii.gz -wild *.nii.gz
rm $datafld/aligned/sub-$SUB/*.xfm

## Step 3: Align each volume to average

In [ ]:
# align template to get mask
flirt -in $datafld/mask/sub-$SUB/sub-${SUB}_ref_anat_res-0.5x0.5x0.5.nii.gz \
	-ref $datafld/aligned/sub-$SUB/avg_sub-$SUB.nii.gz \
	-omat $datafld/mask/sub-$SUB/mask2avg.mat

In [ ]:
# register mask
flirt -in $datafld/mask/sub-$SUB/sub-${SUB}_ref_anat_mask_res-0.5x0.5x0.5.nii.gz \
	-ref $datafld/aligned/sub-$SUB/avg_sub-$SUB.nii.gz \
	-out $datafld/aligned/sub-$SUB/bm_sub-$SUB.nii.gz \
	-applyxfm -init $datafld/mask/sub-$SUB/mask2avg.mat

In [ ]:
# align each volume rigidly to the average
cd $datafld/normalized/sub-$SUB
for f in *.nii.gz; do
	echo $f
	flirt -refweight $datafld/aligned/sub-$SUB/bm_sub-$SUB.nii.gz -dof 6 \
        -in $f -ref $datafld/aligned/sub-$SUB/avg_sub-$SUB.nii.gz -out $datafld/aligned/sub-$SUB/al_$f 
done

## Step 4: Generate 4d concatenated volume (aligned)

In [ ]:
# generate a 4d volume
mkdir -p $datafld/4d
fslmerge -t $datafld/4d/4d_al_sub-$SUB.nii.gz $datafld/aligned/sub-$SUB/al_*.nii.gz